In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import time

In [48]:
data = pd.read_csv('checkins.dat',sep = '|', skipinitialspace=True).dropna()

c:\users\miair\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                396634 non-null  object 
 1   user_id           396634 non-null  float64
 2   venue_id          396634 non-null  float64
 3   latitude          396634 non-null  float64
 4   longitude         396634 non-null  float64
 5   created_at        396634 non-null  object 
dtypes: float64(4), object(2)
memory usage: 18.2+ MB


In [50]:
data.iloc[:,5] = pd.to_datetime(data.iloc[:,5])

In [51]:
data.iloc[:,5] = pd.to_numeric(data.iloc[:,5])

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                396634 non-null  object 
 1   user_id           396634 non-null  float64
 2   venue_id          396634 non-null  float64
 3   latitude          396634 non-null  float64
 4   longitude         396634 non-null  float64
 5   created_at        396634 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 19.7+ MB


In [53]:
data_new = data[:100000].iloc[:, 3:5]

In [54]:
data_new.head()

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [20]:
%%time
clustering  = MeanShift(bandwidth=0.1).fit(data_new)    

Wall time: 2min 50s


In [25]:
np.unique(clustering.labels_)

array([   0,    1,    2, ..., 3228, 3229, 3230], dtype=int32)

In [22]:
clustering.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [56]:
%%time
data_new['clusters'] = clustering.fit_predict(data_new)

Wall time: 2min 53s


In [57]:
data_new = data_new.loc[data_new['clusters']>15]

In [58]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56734 entries, 8 to 233798
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   latitude        56734 non-null  float64
 1   longitude       56734 non-null  float64
 2   clusters        56734 non-null  int32  
dtypes: float64(2), int32(1)
memory usage: 1.5 MB


In [45]:
data_new

,latitude,longitude,clusters,distance
8,45.523452,-122.676207,30,126.291139
10,40.764462,-111.904565,65,129.412641
12,32.221743,-110.926479,23,113.285850
27,33.248664,-111.634299,137,176.723560
28,32.840678,-117.258794,237,264.421301
...,...,...,...,...
233770,35.994033,-78.898619,151,170.370162
233781,29.762884,-95.383061,25,98.604911
233789,33.575000,-117.725556,50,127.903504
233794,29.762884,-95.383061,25,98.604911


In [46]:
latitude = [25.867736, 51.503016, 52.378894, 39.366487, -33.868457]
longitude = [-80.324116, -0.075479, 4.885084, 117.036146, 151.205134]


In [62]:
def distance(X, a, b):
    return np.sqrt((X.iloc[:,1]-a)**2 + (X.iloc[:,2]-b)**2)

Имеется 5 центров с определенными координатами, найти 20 кластерв, находящихся ближе всего к каждому из центров


In [71]:
for i in range(1, 6):
    data_new['distance_{}'.format(i)] = distance(data_new, latitude[i-1], longitude[i-1])

<ipython-input-71-1f9fe3ae75b0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new['distance_{}'.format(i)] = distance(data_new, latitude[i-1], longitude[i-1])


In [99]:
data_new.head()

,latitude,longitude,clusters,distance_1,distance_2,distance_3,distance_4,distance_5
8,45.523452,-122.676207,30,185.031656,176.756715,176.847526,183.937830,150.258115
10,40.764462,-111.904565,65,200.250607,175.888759,174.936726,159.970909,116.279660
12,32.221743,-110.926479,23,171.430832,164.060411,164.307015,177.287260,149.581065
27,33.248664,-111.634299,137,257.170335,213.080902,210.605505,152.314782,79.052590
28,32.840678,-117.258794,237,348.108888,291.007442,287.496573,197.288633,119.644086


In [89]:
for i in range(1, 6):
    print(data_new.loc[data_new['distance_{}'.format(i)] == data_new['distance_{}'.format(i)].min() ].min())

latitude           -6.195438
longitude         106.743783
clusters           33.000000
distance_1        139.223885
distance_2         64.385788
distance_3         61.204491
distance_4        107.711531
distance_5        183.696096
dtype: float64
latitude           -6.195438
longitude         106.743783
clusters           33.000000
distance_1        139.223885
distance_2         64.385788
distance_3         61.204491
distance_4        107.711531
distance_5        183.696096
dtype: float64
latitude           -6.195438
longitude         106.743783
clusters           33.000000
distance_1        139.223885
distance_2         64.385788
distance_3         61.204491
distance_4        107.711531
distance_5        183.696096
dtype: float64
latitude           -0.301226
longitude          36.522964
clusters          103.000000
distance_1        183.633508
distance_2        104.158323
distance_3         99.387863
distance_4         14.321278
distance_5         85.315222
dtype: float64
latitude    